In [179]:
import pandas as pd
import json

# Read the JSON file line by line (json lines format)
with open('data/matches/all_matches_from_tournaments.json', 'r', encoding='utf-8') as f:
	lines = f.readlines()
	data = [json.loads(line) for line in lines]

df = pd.DataFrame(data)

In [180]:
pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows', 150)

In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1553 entries, 0 to 1552
Data columns (total 58 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   version                  1553 non-null   int64  
 1   match_id                 1553 non-null   int64  
 2   draft_timings            1553 non-null   object 
 3   teamfights               1553 non-null   object 
 4   objectives               1553 non-null   object 
 5   chat                     1553 non-null   object 
 6   radiant_gold_adv         1553 non-null   object 
 7   radiant_xp_adv           1553 non-null   object 
 8   cosmetics                1553 non-null   object 
 9   players                  1553 non-null   object 
 10  leagueid                 1553 non-null   int64  
 11  start_time               1553 non-null   int64  
 12  duration                 1553 non-null   int64  
 13  series_id                1553 non-null   int64  
 14  series_type             

Größe von unnested Daten

with open('data/matches/all_matches_from_tournaments.json', 'r', encoding='utf-8') as f:
	lines = f.readlines()
	data = [json.loads(line) for line in lines]

# This will flatten nested dicts, but not lists-of-dicts
df_flat = pd.json_normalize(data, sep='.')

def recursive_json_normalize(data, sep='.'):
	# If data is a list of dicts, flatten each and concat
	if isinstance(data, list):
		df = pd.json_normalize(data, sep=sep)
		for col in df.columns:
			# If column contains lists of dicts, flatten further
			if df[col].apply(lambda x: isinstance(x, list) and len(x) > 0 and isinstance(x[0], dict)).any():
				# Explode and flatten
				df = df.explode(col)
				nested = pd.json_normalize(df[col].dropna(), sep=sep)
				nested.index = df[col].dropna().index
				nested = nested.add_prefix(f"{col}{sep}")
				df = df.drop(columns=[col]).join(nested)
		return df
	else:
		return pd.json_normalize(data, sep=sep)

df_fully_flat = recursive_json_normalize(data)
print(df_fully_flat.info())

In [182]:
import subprocess

def run_get_glicko_for_dates(dates):
	for date in dates:
		# Convert date to string if it's a datetime.date object
		date_str = str(date)
		subprocess.run(['python', 'get_glicko.py', date_str])

run_get_glicko_for_dates(df['tournament_start_date'].unique())

In [183]:
df_players = df[['match_id', 'start_time', 'players', 'tournament_start_date', 'radiant_team_id', 'dire_team_id']].explode('players').reset_index(drop=True)
df_players = pd.concat(
	[df_players[['match_id', 'start_time', 'tournament_start_date', 'radiant_team_id', 'dire_team_id']], pd.DataFrame(df_players['players'].tolist())],
	axis=1
)

In [184]:
df_players['tournament_start_date']

0        2024-09-04
1        2024-09-04
2        2024-09-04
3        2024-09-04
4        2024-09-04
            ...    
15525    2025-07-08
15526    2025-07-08
15527    2025-07-08
15528    2025-07-08
15529    2025-07-08
Name: tournament_start_date, Length: 15530, dtype: object

In [185]:
df_players.rename(columns={'start_time': 'match_start_time'}, inplace=True)

In [186]:
df_players.head(1)

,match_id,match_start_time,tournament_start_date,radiant_team_id,dire_team_id,player_slot,obs_placed,sen_placed,creeps_stacked,camps_stacked,rune_pickups,firstblood_claimed,teamfight_participation,towers_killed,roshans_killed,observers_placed,stuns,max_hero_hit,times,gold_t,lh_t,dn_t,xp_t,obs_log,sen_log,obs_left_log,sen_left_log,purchase_log,kills_log,buyback_log,runes_log,connection_log,lane_pos,obs,sen,actions,pings,purchase,gold_reasons,xp_reasons,killed,item_uses,ability_uses,ability_targets,damage_targets,hero_hits,damage,damage_taken,damage_inflictor,runes,killed_by,kill_streaks,multi_kills,life_state,healing,damage_inflictor_received,randomed,pred_vict,neutral_tokens_log,party_id,permanent_buffs,party_size,account_id,team_number,team_slot,hero_id,hero_variant,item_0,item_1,item_2,item_3,item_4,item_5,backpack_0,backpack_1,...,deaths,assists,leaver_status,last_hits,denies,gold_per_min,xp_per_min,level,net_worth,aghanims_scepter,aghanims_shard,moonshard,hero_damage,tower_damage,hero_healing,gold,gold_spent,ability_upgrades_arr,personaname,name,last_login,rank_tier,is_subscriber,radiant_win,match_start_time,duration,cluster,lobby_type,game_mode,is_contributor,patch,region,isRadiant,win,lose,total_gold,total_xp,kills_per_min,kda,abandons,neutral_kills,tower_kills,courier_kills,lane_kills,hero_kills,observer_kills,sentry_kills,roshan_kills,necronomicon_kills,ancient_kills,buyback_count,observer_uses,sentry_uses,lane_efficiency,lane_efficiency_pct,lane,lane_role,is_roaming,purchase_time,first_purchase_time,item_win,item_usage,purchase_ward_observer,purchase_ward_sentry,actions_per_min,life_state_dead,cosmetics,benchmarks,purchase_tpscroll,purchase_gem,performance_others,purchase_rapier,additional_units,neutral_item_history,item_neutral2
0,7944311818,1726415990,2024-09-04,8599101,2163,0,7,12,6,2,3,0,0.647059,0,0,7,29.466703,"{'type': 'max_hero_hit', 'time': 1029, 'max': ...","[0, 60, 120, 180, 240, 300, 360, 420, 480, 540...","[0, 231, 550, 640, 770, 1113, 1203, 1409, 1580...","[0, 2, 8, 9, 10, 10, 10, 10, 10, 10, 11, 12, 1...","[0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[6, 68, 575, 665, 839, 1085, 1149, 1227, 1846,...","[{'time': -34, 'type': 'obs_log', 'slot': 0, '...","[{'time': 33, 'type': 'sen_log', 'slot': 0, 'x...","[{'time': 326, 'type': 'obs_left_log', 'slot':...","[{'time': 453, 'type': 'sen_left_log', 'slot':...","[{'time': -89, 'key': 'smoke_of_deceit'}, {'ti...","[{'time': 260, 'key': 'npc_dota_hero_dark_will...","[{'time': 1358, 'slot': 0, 'type': 'buyback_lo...","[{'time': 211, 'key': 5}, {'time': 841, 'key':...",[],"{'76': {'76': 4}, '77': {'77': 1, '80': 1}, '7...","{'80': {'151': 1}, '98': {'157': 1}, '108': {'...","{'72': {'127': 1}, '79': {'150': 1}, '81': {'1...","{'1': 4405, '2': 25, '3': 6, '4': 425, '5': 27...",63.0,"{'ward_dispenser': 11, 'smoke_of_deceit': 3, '...","{'0': 659, '1': -670, '6': 350, '11': 90, '12'...","{'0': 135, '1': 3606, '2': 7415, '4': 1960}","{'npc_dota_courier': 1, 'npc_dota_creep_badguy...","{'smoke_of_deceit': 2, 'ward_dispenser': 21, '...","{'ability_lamp_use': 10, 'marci_grapple': 17, ...",{'marci_grapple': {'npc_dota_hero_dark_willow'...,{'marci_grapple': {'npc_dota_hero_dark_willow'...,"{'marci_grapple': 17, 'null': 90, 'marci_compa...","{'npc_dota_hero_dark_willow': 1260, 'npc_dota_...","{'npc_dota_hero_dark_willow': 3886, 'npc_dota_...","{'marci_grapple': 793, 'null': 7038, 'marci_co...","{'5': 1, '8': 2}","{'npc_dota_hero_visage': 2, 'npc_dota_hero_puc...",{},{},"{'0': 1890, '1': 12, '2': 167}","{'npc_dota_hero_marci': 10478, 'npc_dota_hero_...","{'null': 11767, 'dark_willow_bramble_maze': 82...",False,False,[],0,[],10,91730177,0,0,136,1,229,29,73,11,73,73,0,0,...,7,8,0,36,2,248,379,15,5746,0,0,0,10715,0,8182,1031,6835,"[8192, 8235, 1426, 8235, 8235, 8198, 8235, 142...",Seleri,Seleri,2020-10-05T18:16:42.773Z,80.0,False,False,1726415990,2068,187,1,2,False,56,8.0,True,0,1,8547,13062,0.087041,1.38,0,2,0,1,24,3,1,4,0,0,0,2,5,4,0.418957,41.0,1,1,False,"{'smoke_of_de

In [187]:
df_players_name_none_unique = df_players[df_players['name'].isna() & df_players['account_id'].notna()]
df_players_name_none_unique = df_players_name_none_unique['account_id'].unique()

In [188]:
df_players_name_none_unique.tolist()

[164962869,
 116585378,
 369342470,
 171097887,
 115464954,
 48823667,
 208181197,
 181267255,
 120613892,
 140251702,
 295697470,
 910028605,
 114619230,
 296702734,
 276449590,
 1048617659,
 94786276,
 1150772339,
 97658618,
 137193239,
 223342537,
 117514269,
 399804216,
 238239590,
 439345730,
 392006194,
 333351042,
 200573858,
 9403474]

164962869 Shad
116585378 MidOne
369342470 SoNNeikO
171097887 dEsire
115464954 Nande
48823667 Bengan
208181197 Adzantick
181267255 OneJey
120613892 W1sh-
140251702 Mirage`
295697470 Immersion
94786276 Nine
1150772339 mangekyou
97658618 Timado
137193239 Paparazi灬
223342537 lupsione
117514269 laise
399804216 Serenada
238239590 Bryle
439345730 reibl
392006194 ififall
333351042 mrls
9403474 yamich

In [189]:
# Mapping from account_id to player name
account_id_to_name = {
	164962869: "Shad",
	116585378: "MidOne",
	369342470: "SoNNeikO",
	171097887: "dEsire",
	115464954: "Nande",
	48823667: "Bengan",
	208181197: "Adzantick",
	181267255: "OneJey",
	120613892: "W1sh-",
	140251702: "Mirage",
	295697470: "Immersion",
	94786276: "Nine",
	1150772339: "mangekyou",
	97658618: "Timado",
	137193239: "Paparazi灬",
	223342537: "lupsione",
	117514269: "laise",
	399804216: "Serenada",
	238239590: "Bryle",
	439345730: "reibl",
	392006194: "ififall",
	333351042: "mrls",
	9403474: "yamich",
	910028605: "Unknown",
	114619230: "Unknown",
	296702734: "Unknown",
	276449590: "Unknown",
	1048617659: "Unknown",
	200573858: "Unknown"
}

# Update player names in df_players where account_id matches
df_players.loc[df_players['account_id'].isin(account_id_to_name.keys()), 'name'] = \
	df_players.loc[df_players['account_id'].isin(account_id_to_name.keys()), 'account_id'].map(account_id_to_name)

In [190]:
df_players = df_players[df_players['name'] != 'Unknown'].copy()

In [191]:
df_players

,match_id,match_start_time,tournament_start_date,radiant_team_id,dire_team_id,player_slot,obs_placed,sen_placed,creeps_stacked,camps_stacked,rune_pickups,firstblood_claimed,teamfight_participation,towers_killed,roshans_killed,observers_placed,stuns,max_hero_hit,times,gold_t,lh_t,dn_t,xp_t,obs_log,sen_log,obs_left_log,sen_left_log,purchase_log,kills_log,buyback_log,runes_log,connection_log,lane_pos,obs,sen,actions,pings,purchase,gold_reasons,xp_reasons,killed,item_uses,ability_uses,ability_targets,damage_targets,hero_hits,damage,damage_taken,damage_inflictor,runes,killed_by,kill_streaks,multi_kills,life_state,healing,damage_inflictor_received,randomed,pred_vict,neutral_tokens_log,party_id,permanent_buffs,party_size,account_id,team_number,team_slot,hero_id,hero_variant,item_0,item_1,item_2,item_3,item_4,item_5,backpack_0,backpack_1,...,deaths,assists,leaver_status,last_hits,denies,gold_per_min,xp_per_min,level,net_worth,aghanims_scepter,aghanims_shard,moonshard,hero_damage,tower_damage,hero_healing,gold,gold_spent,ability_upgrades_arr,personaname,name,last_login,rank_tier,is_subscriber,radiant_win,match_start_time,duration,cluster,lobby_type,game_mode,is_contributor,patch,region,isRadiant,win,lose,total_gold,total_xp,kills_per_min,kda,abandons,neutral_kills,tower_kills,courier_kills,lane_kills,hero_kills,observer_kills,sentry_kills,roshan_kills,necronomicon_kills,ancient_kills,buyback_count,observer_uses,sentry_uses,lane_efficiency,lane_efficiency_pct,lane,lane_role,is_roaming,purchase_time,first_purchase_time,item_win,item_usage,purchase_ward_observer,purchase_ward_sentry,actions_per_min,life_state_dead,cosmetics,benchmarks,purchase_tpscroll,purchase_gem,performance_others,purchase_rapier,additional_units,neutral_item_history,item_neutral2
0,7944311818,1726415990,2024-09-04,8599101,2163,0,7,12,6,2,3,0,0.647059,0,0,7,29.466703,"{'type': 'max_hero_hit', 'time': 1029, 'max': ...","[0, 60, 120, 180, 240, 300, 360, 420, 480, 540...","[0, 231, 550, 640, 770, 1113, 1203, 1409, 1580...","[0, 2, 8, 9, 10, 10, 10, 10, 10, 10, 11, 12, 1...","[0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[6, 68, 575, 665, 839, 1085, 1149, 1227, 1846,...","[{'time': -34, 'type': 'obs_log', 'slot': 0, '...","[{'time': 33, 'type': 'sen_log', 'slot': 0, 'x...","[{'time': 326, 'type': 'obs_left_log', 'slot':...","[{'time': 453, 'type': 'sen_left_log', 'slot':...","[{'time': -89, 'key': 'smoke_of_deceit'}, {'ti...","[{'time': 260, 'key': 'npc_dota_hero_dark_will...","[{'time': 1358, 'slot': 0, 'type': 'buyback_lo...","[{'time': 211, 'key': 5}, {'time': 841, 'key':...",[],"{'76': {'76': 4}, '77': {'77': 1, '80': 1}, '7...","{'80': {'151': 1}, '98': {'157': 1}, '108': {'...","{'72': {'127': 1}, '79': {'150': 1}, '81': {'1...","{'1': 4405, '2': 25, '3': 6, '4': 425, '5': 27...",63.0,"{'ward_dispenser': 11, 'smoke_of_deceit': 3, '...","{'0': 659, '1': -670, '6': 350, '11': 90, '12'...","{'0': 135, '1': 3606, '2': 7415, '4': 1960}","{'npc_dota_courier': 1, 'npc_dota_creep_badguy...","{'smoke_of_deceit': 2, 'ward_dispenser': 21, '...","{'ability_lamp_use': 10, 'marci_grapple': 17, ...",{'marci_grapple': {'npc_dota_hero_dark_willow'...,{'marci_grapple': {'npc_dota_hero_dark_willow'...,"{'marci_grapple': 17, 'null': 90, 'marci_compa...","{'npc_dota_hero_dark_willow': 1260, 'npc_dota_...","{'npc_dota_hero_dark_willow': 3886, 'npc_dota_...","{'marci_grapple': 793, 'null': 7038, 'marci_co...","{'5': 1, '8': 2}","{'npc_dota_hero_visage': 2, 'npc_dota_hero_puc...",{},{},"{'0': 1890, '1': 12, '2': 167}","{'npc_dota_hero_marci': 10478, 'npc_dota_hero_...","{'null': 11767, 'dark_willow_bramble_maze': 82...",False,False,[],0,[],10,91730177,0,0,136,1,229,29,73,11,73,73,0,0,...,7,8,0,36,2,248,379,15,5746,0,0,0,10715,0,8182,1031,6835,"[8192, 8235, 1426, 8235, 8235, 8198, 8235, 142...",Seleri,Seleri,2020-10-05T18:16:42.773Z,80.0,False,False,1726415990,2068,187,1,2,False,56,8.0,True,0,1,8547,13062,0.087041,1.38,0,2,0,1,24,3,1,4,0,0,0,2,5,4,0.418957,41.0,1,1,False,"{'smoke_of_de

In [192]:
# df_players[df_players['account_id'].isin(df_players_name_none_unique.tolist())][['account_id', 'match_id', 'player_slot', 'gold_per_min', 'xp_per_min']].drop_duplicates('account_id')

To do:
- Fix missing player info like in exploration.ipynb afer full dataset is available.

## Enrich playerdata with fantasy role

In [193]:
with open('data/proPlayers.json', 'r', encoding='utf-8') as f:
	pro_players_data = json.load(f)

df_fantasy_role = pd.DataFrame(pro_players_data)[['account_id', 'fantasy_role']]


In [194]:
df_players = df_players.merge(df_fantasy_role, on='account_id', how='left')


In [195]:
player_data_filter = [
    # Metadata
	'match_id',
    'match_start_time',
	'account_id',
    'name',
	'fantasy_role',
    'hero_id',
    'hero_variant',
    'isRadiant',
    'radiant_team_id',
	'dire_team_id',
	'tournament_start_date',
    'duration',
    'win',
    # FP Stats
	'kills',
	'deaths',
    'last_hits',
	'denies',
    'gold_per_min',
	'towers_killed',
	'roshans_killed',
	'teamfight_participation',
    'obs_placed',
	'camps_stacked',
	'rune_pickups',
	'firstblood_claimed',
	'stuns',
    # Additional Stats
	'creeps_stacked',
    'assists',
	'xp_per_min',
    'lane_efficiency',
	# Feature Engineering
	
]

In [196]:
df_players = df_players[player_data_filter]

In [197]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15437 entries, 0 to 15436
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15437 non-null  int64  
 1   match_start_time         15437 non-null  int64  
 2   match_start_time         15437 non-null  int64  
 3   account_id               15437 non-null  int64  
 4   name                     15437 non-null  object 
 5   fantasy_role             14666 non-null  float64
 6   hero_id                  15437 non-null  int64  
 7   hero_variant             15437 non-null  int64  
 8   isRadiant                15437 non-null  bool   
 9   radiant_team_id          15437 non-null  int64  
 10  dire_team_id             15437 non-null  int64  
 11  tournament_start_date    15437 non-null  object 
 12  duration                 15437 non-null  int64  
 13  win                      15437 non-null  int64  
 14  kills                 

In [198]:
df_players.head(2)

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency
0,7944311818,1726415990,1726415990,91730177,Seleri,2.0,136,1,True,8599101,2163,2024-09-04,2068,0,3,7,36,2,248,0,0,0.647059,7,2,3,0,29.466703,6,8,379,0.418957
1,7944311818,1726415990,1726415990,116934015,dyrachyo,1.0,21,3,True,8599101,2163,2024-09-04,2068,0,4,7,275,7,540,0,0,0.705882,0,7,0,0,71.866130,22,8,527,0.618634


In [199]:
# Enrich player data with team_id based on isRadiant flag
df_players['team_id'] = df_players.apply(lambda row: row['radiant_team_id'] if row['isRadiant'] else row['dire_team_id'], axis=1)
df_players['enemy_team_id'] = df_players.apply(lambda row: row['dire_team_id'] if row['isRadiant'] else row['radiant_team_id'], axis=1)



In [200]:
def assign_glicko_rating(df: pd.DataFrame, date: str):
	with open(f'data/ratings/ratings_{date}.json', 'r', encoding='utf-8') as f:
		glicko_ratings = pd.DataFrame(json.load(f))
	df = df[df['tournament_start_date'] == date].copy()
	df['team_id'] = df['team_id'].astype('Int64').astype(str)
	df['enemy_team_id'] = df['enemy_team_id'].astype('Int64').astype(str)
	glicko_ratings['valveId'] = glicko_ratings['valveId'].astype('Int64').astype(str)
	df = df.merge(glicko_ratings, left_on='team_id', right_on='valveId', how='left', suffixes=('', '_team'))
	df = df.merge(glicko_ratings, left_on='enemy_team_id', right_on='valveId', how='left', suffixes=('', '_enemy'))
	# Rename the columns from the second merge with prefix 'enemy_'
	for col in ['teamName', 'valveId', 'glicko2_rating']:
		df.rename(columns={f'{col}_enemy': f'enemy_{col}'}, inplace=True)
	return df

In [201]:
# Collect all unique dates
unique_dates = df_players['tournament_start_date'].unique()

# List to store the resulting dataframes
df_glicko_list = []

for date in unique_dates:
	df_glicko = assign_glicko_rating(df_players, date)
	df_glicko_list.append(df_glicko)

# Concatenate all dataframes into one
df_players_with_glicko = pd.concat(df_glicko_list, ignore_index=True)

In [202]:
df_players_with_glicko.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15437 entries, 0 to 15436
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15437 non-null  int64  
 1   match_start_time         15437 non-null  int64  
 2   match_start_time         15437 non-null  int64  
 3   account_id               15437 non-null  int64  
 4   name                     15437 non-null  object 
 5   fantasy_role             14666 non-null  float64
 6   hero_id                  15437 non-null  int64  
 7   hero_variant             15437 non-null  int64  
 8   isRadiant                15437 non-null  bool   
 9   radiant_team_id          15437 non-null  int64  
 10  dire_team_id             15437 non-null  int64  
 11  tournament_start_date    15437 non-null  object 
 12  duration                 15437 non-null  int64  
 13  win                      15437 non-null  int64  
 14  kills                 

Drop data that can not be assigned a Glicko Score.

In [203]:
#df_players_with_glicko.dropna(axis=0, inplace=True)

In [204]:
df_players_with_glicko[df_players_with_glicko['valveId'].isna()]

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating
1440,7982874526,1728650424,1728650424,96803083,Stormstormer,4.0,74,2,True,9498970,8254400,2024-10-04,1945,0,2,6,170,19,381,0,0,0.888889,2,0,4,0,32.135303,0,6,399,0.791835,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
1441,7982874526,1728650424,1728650424,369342470,SoNNeikO,NaN,5,1,True,9498970,8254400,2024-10-04,1945,0,1,8,30,1,232,0,0,0.444444,11,1,1,0,57.199802,3,3,247,0.265764,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
1442,7982874526,1728650424,1728650424,155162307,Smiling Knight,NaN,70,2,True,9498970,8254400,2024-10-04,1945,0,1,4,290,4,527,0,0,0.222222,0,1,5,0,0.000000,3,1,556,0.741512,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
1443,7982874526,1728650424,1728650424,230487729,Ekki,2.0,73,1,True,9498970,8254400,2024-10-04,1945,0,1,7,107,4,313,0,0,0.555556,3,4,2,0,50.566140,10,4,305,0.416330,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
1444,7982874526,1728650424,1728650424,50580004,Xibbe,1.0,92,3,True,9498970,8254400,2024-10-04,1945,0,4,8,128,9,354,1,0,0.777778,0,2,2,1,24.031889,8,3,318,0.612369,9498970,8254400,NaN,NaN,NaN,beastcoast,8254400,1717.018626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15427,8366140862,1751966193,1751966193,957204049,gotthejuice,1.0,102,1,True,9828954,8597976,2025-07-08,3080,1,13,3,669,16,901,1,1,0.838710,0,2,3,0,35.401466,3,13,1134,0.785974,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231
15428,8366140862,1751966193,1751966193,185590374,Niku,1.0,11,2,True,9828954,8597976,2025-07-08,3080,1,12,6,563,9,781,1,0,0.806452,1,2,11,1,34.614555,5,13,1103,0.852061,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231
15429,8366140862,1751966193,1751966193,835864135,pma,1.0,120,1,True,9828954,8597976,2025-07-08,3080,1,5,3,442,4,639,0,1,0.935484,0,4,1,0,8.685791,7,24,1038,0.604285,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231
15430,8366140862,1751966193,1751966193,111030315,KG_Zayac,2.0,27,2,True,9828954,8597976,2025-07-08,3080,1,0,10,72,0,314,4,0,0.580645,9,8,3,0,124.567920,35,18,460,0.268998,9828954,8597976,NaN,NaN,NaN,Talon,8597976,1799.656231


In [205]:
df_players_with_glicko.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15437 entries, 0 to 15436
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15437 non-null  int64  
 1   match_start_time         15437 non-null  int64  
 2   match_start_time         15437 non-null  int64  
 3   account_id               15437 non-null  int64  
 4   name                     15437 non-null  object 
 5   fantasy_role             14666 non-null  float64
 6   hero_id                  15437 non-null  int64  
 7   hero_variant             15437 non-null  int64  
 8   isRadiant                15437 non-null  bool   
 9   radiant_team_id          15437 non-null  int64  
 10  dire_team_id             15437 non-null  int64  
 11  tournament_start_date    15437 non-null  object 
 12  duration                 15437 non-null  int64  
 13  win                      15437 non-null  int64  
 14  kills                 

In [206]:
df_players_with_glicko.head(11)

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating
0,7944311818,1726415990,1726415990,91730177,Seleri,2.0,136,1,True,8599101,2163,2024-09-04,2068,0,3,7,36,2,248,0,0,0.647059,7,2,3,0,29.466703,6,8,379,0.418957,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
1,7944311818,1726415990,1726415990,116934015,dyrachyo,1.0,21,3,True,8599101,2163,2024-09-04,2068,0,4,7,275,7,540,0,0,0.705882,0,7,0,0,71.866130,22,8,527,0.618634,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
2,7944311818,1726415990,1726415990,97590558,Ace ♠,1.0,108,2,True,8599101,2163,2024-09-04,2068,0,2,5,208,5,408,1,0,0.647059,0,2,1,0,104.965775,4,9,499,0.942603,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
3,7944311818,1726415990,1726415990,16497807,tOfu,2.0,9,2,True,8599101,2163,2024-09-04,2068,0,4,8,125,0,312,0,0,0.647059,7,5,6,0,31.299086,10,7,378,0.529709,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
4,7944311818,1726415990,1726415990,221666230,Quinn,4.0,120,1,True,8599101,2163,2024-09-04,2068,0,4,7,207,7,443,0,0,0.705882,1,0,5,0,0.319238,0,8,614,0.840137,8599101,2163,Gaimin Gladiators,8599101,1980.466489,Team Liquid,2163,1959.607080
5,7944311818,1726415990,1726415990,54580962,Insania,2.0,51,2,False,8599101,2163,2024-09-04,2068,1,1,4,47,2,334,0,0,0.764706,7,4,9,0,45.298760,12,25,454,0.336904,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489
6,7944311818,1726415990,1726415990,86698277,33,1.0,92,3,False,8599101,2163,2024-09-04,2068,1,6,3,127,11,571,4,0,0.676471,0,4,4,1,40.065290,14,17,610,0.799313,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489
7,7944311818,1726415990,1726415990,201358612,Nisha,4.0,13,1,False,8599101,2163,2024-09-04,2068,1,11,1,299,8,679,1,0,0.852941,2,2,18,0,15.833374,4,18,843,0.854487,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489
8,7944311818,1726415990,1726415990,77490514,Boxi,2.0,119,1,False,8599101,2163,2024-09-04,2068,1,5,4,29,2,356,0,0,0.647059,7,8,4,0,146.532930,44,18,461,0.296079,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489
9,7944311818,1726415990,1726415990,152962063,m1CKe,1.0,53,2,False,8599101,2163,2024-09-04,2068,1,11,5,344,9,810,4,1,0.941176,0,4,1,0,58.966675,10,21,868,0.755053,2163,8599101,Team Liquid,2163,1959.607080,Gaimin Gladiators,8599101,1980.466489


In [207]:
df_players = df_players_with_glicko.copy()

# Fantasy Calculation

In [208]:
def calculate_fantasy_score(row):
	return (
		row['kills'] * 0.3 +
		(3 - row['deaths'] * 0.3) +
		(row['last_hits'] + row['denies']) * 0.003 +
		row['gold_per_min'] * 0.002 +
		row['towers_killed'] +
		row['roshans_killed'] +
		row['teamfight_participation'] * 3 +
		row['obs_placed'] * 0.5 +
		row['camps_stacked'] * 0.5 +
		row['rune_pickups'] * 0.25 +
		row['firstblood_claimed'] * 4 +
		row['stuns'] * 0.05
	)

df_players['fantasy_score'] = df_players.apply(calculate_fantasy_score, axis=1)

In [209]:
df_players.sample(11)

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating,fantasy_score
10871,8208709408,1741623661,1741623661,138177198,Vitaly 誇り,1.0,55,1,False,36,9699533,2025-03-08,2532,0,2,6,274,2,394,0,0,0.437500,0,2,1,0,5.632934,6,5,433,0.738076,9699533,36,NaN,NaN,NaN,Natus Vincere,36,1782.071301,6.260147
9172,8176853300,1739722440,1739722440,101259972,Oli~,2.0,110,1,True,9678064,9303484,2025-02-16,3713,0,3,8,123,1,283,0,0,0.461538,22,6,4,0,2.500000,20,3,383,0.422191,9678064,9303484,NaN,NaN,NaN,HEROIC,9303484,1906.413747,18.947615
2175,7973769812,1728120590,1728120590,177203952,Yuma,1.0,73,1,False,8375259,8728920,2024-10-04,3293,1,7,4,548,2,872,2,1,0.939394,0,6,5,0,82.797676,18,24,1000,1.049313,8728920,8375259,nouns,8728920,1850.163862,Infinity,8375259,1634.483483,21.502066
3834,8019572135,1730739301,1730739301,103735745,Saksa,2.0,100,1,True,8291895,2163,2024-10-27,2097,0,3,6,25,2,237,0,0,0.700000,4,2,5,0,15.900318,6,5,296,0.450889,8291895,2163,Tundra Esports,8291895,2013.159488,Team Liquid,2163,1969.347843,9.800016
4627,8010054526,1730219939,1730219939,165564598,MieRo,1.0,137,1,False,9572001,8255888,2024-10-27,3106,1,6,6,363,7,591,2,0,0.902439,0,2,5,0,70.135600,5,31,791,0.797696,8255888,9572001,BetBoom Team,8255888,1944.623862,NaN,NaN,NaN,15.756097
4434,8011104295,1730290380,1730290380,196878136,Kataomi`,2.0,84,2,True,8255888,8728920,2024-10-27,1847,1,2,3,29,1,297,0,0,0.656250,10,0,4,0,17.267921,0,19,411,0.343775,8255888,8728920,BetBoom Team,8255888,1944.623862,nouns,8728920,1833.880424,12.216146
13688,8302083784,1747844520,1747844520,858106446,kiyotaka,4.0,17,2,True,9467224,8261500,2025-05-19,2379,1,14,7,408,7,757,1,1,0.812500,2,2,15,0,33.500430,9,12,1045,0.850040,9467224,8261500,Aurora Gaming,9467224,1942.349787,Xtreme Gaming,8261500,1880.059434,19.721522
12248,8262062332,1745145396,1745145396,113331514,Miposhka,2.0,3,2,True,7119388,9640842,2025-04-19,2194,1,4,7,33,4,326,1,0,0.714286,10,0,3,0,45.367043,0,16,517,0.304972,7119388,9640842,Team Spirit,7119388,2078.156164,Team Tidebound,9640842,1977.760680,14.024209
5071,8008146416,1730112903,1730112903,96803083,Stormstormer,4.0,17,2,True,9498970,2163,2024-10-27,4063,1,6,10,585,9,614,0,0,0.812500,2,1,22,0,74.400650,2,20,860,0.900162,9498970,2163,AVULUS,9498970,1755.454083,Team Liquid,2163,1969.347843,17.967532
9984,8247075519,1744131601,1744131601,73401082,Dukalis,2.0,64,3,True,9572001,2163,2025-04-07,2609,1,1,7,45,1,331,0,0,0.666667,10,3,6,0,34.066895,10,17,441,0.458367,9572001,2163,PARIVISION,9572001,2079.455191,Team Liquid,2163,2015.457255,13.703345


In [210]:
# Create feature showing the rating advantage of the player
df_players['rating_advantage'] = df_players['glicko2_rating'] - df_players['enemy_glicko2_rating']

In [211]:
# Create feature to make hero_variant useable
# Das muss gemacht werden, damit der Kontext von hero + facet korrekt ist.
df_players['hero_variant'] = df_players['hero_id'].astype(str) + ", " + df_players['hero_variant'].astype(str)

In [212]:
df_players['fantasy_role'].value_counts()

1.0    5830
2.0    5741
4.0    3057
0.0      38
Name: fantasy_role, dtype: int64

In [213]:
df_players[df_players['name'] == 'yamich']


,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating,fantasy_score,rating_advantage
12716,8313079694,1748540520,1748540520,9403474,yamich,NaN,119,"119, 1",False,9467224,7554697,2025-05-19,1875,1,10,4,41,0,362,0,0,0.666667,4,0,3,0,132.434510,0,16,485,0.560631,7554697,9467224,Nigma Galaxy,7554697,1882.868495,Aurora Gaming,9467224,1942.349787,17.018726,-59.481293
12721,8313000571,1748536260,1748536260,9403474,yamich,NaN,65,"65, 1",True,7554697,9467224,2025-05-19,2344,0,0,7,105,2,279,0,0,0.764706,6,5,2,0,30.199707,12,13,398,0.334883,7554697,9467224,Nigma Galaxy,7554697,1882.868495,Aurora Gaming,9467224,1942.349787,11.583103,-59.481293
12736,8312893959,1748532300,1748532300,9403474,yamich,NaN,65,"65, 1",False,9467224,7554697,2025-05-19,1815,1,4,5,118,1,404,0,0,0.689655,4,15,4,0,29.066162,54,16,559,0.378941,7554697,9467224,Nigma Galaxy,7554697,1882.868495,Aurora Gaming,9467224,1942.349787,17.887274,-59.481293
12781,8312016561,1748467080,1748467080,9403474,yamich,NaN,93,"93, 2",True,7554697,9572001,2025-05-19,2247,0,2,7,56,4,226,0,0,0.470588,9,3,6,0,0.000000,8,6,302,0.358124,7554697,9572001,Nigma Galaxy,7554697,1882.868495,PARIVISION,9572001,2098.650175,11.043765,-215.781680
12796,8311967971,1748463433,1748463433,9403474,yamich,NaN,65,"65, 1",False,9572001,7554697,2025-05-19,1765,0,3,6,65,3,269,0,0,0.533333,4,10,1,0,26.732666,27,5,383,0.272231,7554697,9572001,Nigma Galaxy,7554697,1882.868495,PARIVISION,9572001,2098.650175,13.028633,-215.781680
12916,8310581647,1748371981,1748371981,9403474,yamich,NaN,100,"100, 1",False,2163,7554697,2025-05-19,1823,0,0,11,37,1,174,1,0,0.500000,3,1,2,0,21.066700,2,4,298,0.242724,7554697,2163,Nigma Galaxy,7554697,1882.868495,Team Liquid,2163,2003.908505,6.215335,-121.040011
12921,8310500064,1748368202,1748368202,9403474,yamich,NaN,65,"65, 1",True,7554697,2163,2025-05-19,1967,1,9,1,129,5,429,0,0,0.684211,5,13,4,0,46.732910,45,17,607,0.378537,7554697,2163,Nigma Galaxy,7554697,1882.868495,Team Liquid,2163,2003.908505,21.049277,-121.040011
12931,8310411911,1748364242,1748364242,9403474,yamich,NaN,27,"27, 2",True,7554697,2163,2025-05-19,2097,0,0,6,30,4,173,0,0,0.500000,6,6,0,0,64.634650,27,2,227,0.307599,7554697,2163,Nigma Galaxy,7554697,1882.868495,Team Liquid,2163,2003.908505,12.379732,-121.040011
13046,8308969352,1748272146,1748272146,9403474,yamich,NaN,65,"65, 1",False,8255888,7554697,2025-05-19,2872,0,7,11,116,0,317,0,0,0.760000,4,12,8,1,58.799667,31,12,377,0.517987,7554697,8255888,Nigma Galaxy,7554697,1882.868495,BetBoom Team,8255888,1997.124150,22.001983,-114.255656
13056,8308860629,1748268186,1748268186,9403474,yamich,NaN,7,"7, 3",False,8255888,7554697,2025-05-19,2149,0,2,5,73,0,291,0,0,0.666667,5,2,3,0,102.072395,6,8,365,0.368836,7554697,8255888,Nigma Galaxy,7554697,1882.868495,BetBoom Team,8255888,1997.124150,14.254620,-114.255656


In [214]:
df_players[df_players['fantasy_role'] == 0]

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating,fantasy_score,rating_advantage
1656,7979879905,1728461820,1728461820,489696354,Han,0.0,107,"107, 1",False,8254400,8574561,2024-10-04,3283,0,4,4,380,7,539,0,0,0.807692,1,1,33,0,23.599426,3,17,657,0.703719,8574561,8254400,Azure Ray,8574561,1849.090924,beastcoast,8254400,1717.018626,18.092048,132.072298
1661,7979825770,1728457212,1728457212,489696354,Han,0.0,52,"52, 2",True,8574561,8254400,2024-10-04,2928,0,10,10,473,16,626,0,0,0.785714,0,0,12,0,71.401085,0,12,656,0.853476,8574561,8254400,Azure Ray,8574561,1849.090924,beastcoast,8254400,1717.018626,14.646197,132.072298
1706,7978651098,1728386377,1728386377,489696354,Han,0.0,39,"39, 2",False,8291895,8574561,2024-10-04,1161,0,3,1,141,4,482,0,0,0.666667,1,0,12,0,0.000000,0,1,526,0.809620,8574561,8291895,Azure Ray,8574561,1849.090924,Tundra Esports,8291895,1997.451258,10.499000,-148.360334
1711,7978584525,1728382231,1728382231,489696354,Han,0.0,16,"16, 2",True,8574561,8291895,2024-10-04,2520,0,4,11,297,3,445,0,0,0.818182,0,2,13,0,50.400330,4,5,503,0.824171,8574561,8291895,Azure Ray,8574561,1849.090924,Tundra Esports,8291895,1997.451258,11.914562,-148.360334
1971,7975493346,1728206355,1728206355,489696354,Han,0.0,106,"106, 1",True,8574561,7554697,2024-10-04,2559,1,6,3,253,12,550,0,0,0.750000,1,0,21,0,83.431330,0,19,762,0.757478,8574561,7554697,Azure Ray,8574561,1849.090924,Nigma Galaxy,7554697,1818.833431,17.966566,30.257492
1996,7975410015,1728202249,1728202249,489696354,Han,0.0,25,"25, 1",False,7554697,8574561,2024-10-04,2326,0,14,5,352,17,638,0,0,0.850000,1,0,8,0,45.067870,0,3,719,0.947454,8574561,7554697,Azure Ray,8574561,1849.090924,Nigma Galaxy,7554697,1818.833431,15.386393,30.257492
2021,7975331380,1728198057,1728198057,489696354,Han,0.0,25,"25, 1",True,8574561,7554697,2024-10-04,2495,1,6,2,439,9,749,1,1,0.722222,1,3,14,1,25.666927,6,20,995,0.949070,8574561,7554697,Azure Ray,8574561,1849.090924,Nigma Galaxy,7554697,1818.833431,21.992013,30.257492
2162,7973809955,1728122705,1728122705,489696354,Han,0.0,35,"35, 1",True,8574561,9303484,2024-10-04,1547,0,4,5,179,5,442,0,0,0.900000,2,1,1,0,0.066631,4,5,430,0.727162,8574561,9303484,Azure Ray,8574561,1849.090924,HEROIC,9303484,1883.953464,8.589332,-34.862540
2186,7973734261,1728119042,1728119042,489696354,Han,0.0,74,"74, 2",False,9303484,8574561,2024-10-04,1931,0,6,6,157,7,397,0,0,0.777778,0,1,7,0,48.565575,3,8,477,0.644705,8574561,9303484,Azure Ray,8574561,1849.090924,HEROIC,9303484,1883.953464,11.297612,-34.862540
2376,7972143214,1728033019,1728033019,489696354,Han,0.0,74,"74, 2",False,8375259,8574561,2024-10-04,2226,1,14,2,161,7,596,4,0,0.772727,1,0,10,0,95.634254,0,20,790,0.753840,8574561,8375259,Azure Ray,8574561,1849.090924,Infinity,8375259,1634.483483,22.395894,214.607440


In [215]:
df_players[df_players['fantasy_role'] != df_players['fantasy_role']]

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating,fantasy_score,rating_advantage
1255,7986033321,1728816842,1728816842,164962869,Shad,NaN,11,"11, 1",False,9303484,8291895,2024-10-04,2630,0,7,6,476,9,653,4,1,0.360000,0,0,4,0,4.083203,0,2,715,0.755255,8291895,9303484,Tundra Esports,8291895,1997.451258,HEROIC,9303484,1883.953464,13.345160,113.497793
1259,7986033321,1728816842,1728816842,116585378,MidOne,NaN,61,"61, 2",False,9303484,8291895,2024-10-04,2630,0,5,6,365,11,576,3,0,0.800000,1,0,10,0,11.200000,0,15,686,0.901172,8291895,9303484,Tundra Esports,8291895,1997.451258,HEROIC,9303484,1883.953464,13.940000,113.497793
1260,7985929572,1728812025,1728812025,164962869,Shad,NaN,72,"72, 1",True,8291895,9303484,2024-10-04,3017,1,9,1,559,14,797,5,2,0.575000,0,4,6,0,33.967090,11,14,986,0.795877,8291895,9303484,Tundra Esports,8291895,1997.451258,HEROIC,9303484,1883.953464,22.636355,113.497793
1264,7985929572,1728812025,1728812025,116585378,MidOne,NaN,106,"106, 2",True,8291895,9303484,2024-10-04,3017,1,9,4,330,9,611,1,0,0.750000,0,2,12,0,151.596830,5,21,827,0.736055,8291895,9303484,Tundra Esports,8291895,1997.451258,HEROIC,9303484,1883.953464,21.568842,113.497793
1275,7985816345,1728806581,1728806581,164962869,Shad,NaN,94,"94, 3",False,9303484,8291895,2024-10-04,3648,0,5,4,865,1,784,1,1,0.640000,0,2,8,0,53.901142,6,11,853,0.791835,8291895,9303484,Tundra Esports,8291895,1997.451258,HEROIC,9303484,1883.953464,17.081057,113.497793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15190,8367682559,1752069781,1752069781,417235485,prblms,NaN,138,"138, 1",True,9823272,9729720,2025-07-08,1955,1,9,0,73,5,423,0,0,0.764706,5,4,2,1,49.266846,15,17,491,0.596807,9823272,9729720,NaN,NaN,NaN,Virtus.pro,9729720,1694.840836,20.537460,NaN
15289,8366846045,1752003122,1752003122,97658618,Timado,NaN,54,"54, 5",True,39,2163,2025-07-08,1920,0,2,2,251,23,484,0,0,0.272727,0,2,2,0,3.800000,2,1,580,0.753032,39,2163,Shopify Rebellion,39,1819.739986,Team Liquid,2163,2018.797697,7.298182,-199.057712
15313,8366766184,1751998503,1751998503,97658618,Timado,NaN,109,"109, 1",False,2163,39,2025-07-08,2364,0,7,2,520,8,758,3,0,0.733333,0,0,3,0,0.000000,0,4,745,0.774454,39,2163,Shopify Rebellion,39,1819.739986,Team Liquid,2163,2018.797697,13.550000,-199.057712
15330,8366657083,1751992681,1751992681,417235485,prblms,NaN,100,"100, 1",True,9823272,9467224,2025-07-08,2512,0,0,10,29,4,256,0,0,0.565217,8,1,1,0,35.067360,3,13,408,0.248585,9823272,9467224,NaN,NaN,NaN,Aurora Gaming,9467224,1975.721422,8.810020,NaN


In [216]:
# There are two players with an invalid fantasy role (0). Manually set them to the correct role.
df_players.loc[df_players['account_id'] == 1657135701, 'fantasy_role'] = 1 # nesfeer
df_players.loc[df_players['account_id'] == 489696354, 'fantasy_role'] = 4 # Han
df_players.loc[df_players['account_id'] == 208181197, 'fantasy_role'] = 4 # Adzantick
df_players.loc[df_players['account_id'] == 164532005, 'fantasy_role'] = 4 # Armel
df_players.loc[df_players['account_id'] == 48823667, 'fantasy_role'] = 2 # Bengan
df_players.loc[df_players['account_id'] == 90423751, 'fantasy_role'] = 2 # Bignum
df_players.loc[df_players['account_id'] == 238239590, 'fantasy_role'] = 4 # Bryle
df_players.loc[df_players['account_id'] == 160119017, 'fantasy_role'] = 1 # Fayde
df_players.loc[df_players['account_id'] == 295697470, 'fantasy_role'] = 2 # Immersion
df_players.loc[df_players['account_id'] == 104334048, 'fantasy_role'] = 2 # Kidaro
df_players.loc[df_players['account_id'] == 104512126, 'fantasy_role'] = 4 # Mac
df_players.loc[df_players['account_id'] == 116585378, 'fantasy_role'] = 1 # MidOne
df_players.loc[df_players['account_id'] == 140251702, 'fantasy_role'] = 4 # Mirage
df_players.loc[df_players['account_id'] == 115464954, 'fantasy_role'] = 1 # Nande
df_players.loc[df_players['account_id'] == 94786276, 'fantasy_role'] = 4 # Nine
df_players.loc[df_players['account_id'] == 181267255, 'fantasy_role'] = 2 # OneJey
df_players.loc[df_players['account_id'] == 137193239, 'fantasy_role'] = 4 # Paparazi灬
df_players.loc[df_players['account_id'] == 399804216, 'fantasy_role'] = 4 # Serenada
df_players.loc[df_players['account_id'] == 164962869, 'fantasy_role'] = 1 # Shad
df_players.loc[df_players['account_id'] == 155162307, 'fantasy_role'] = 1 # Smiling Knight
df_players.loc[df_players['account_id'] == 369342470, 'fantasy_role'] = 2 # SoNNeikO
df_players.loc[df_players['account_id'] == 97658618, 'fantasy_role'] = 1 # Timado
df_players.loc[df_players['account_id'] == 171097887, 'fantasy_role'] = 1 # Timado
df_players.loc[df_players['account_id'] == 120613892, 'fantasy_role'] = 4 # W1sh-
df_players.loc[df_players['account_id'] == 81475303, 'fantasy_role'] = 1 # Yamsun
df_players.loc[df_players['account_id'] == 392006194, 'fantasy_role'] = 1 # ififall
df_players.loc[df_players['account_id'] == 117514269, 'fantasy_role'] = 1 # laise
df_players.loc[df_players['account_id'] == 223342537, 'fantasy_role'] = 2 # lupsione
df_players.loc[df_players['account_id'] == 1150772339, 'fantasy_role'] = 1 # mangekyou
df_players.loc[df_players['account_id'] == 333351042, 'fantasy_role'] = 2 # mrls
df_players.loc[df_players['account_id'] == 417235485, 'fantasy_role'] = 2 # prblms
df_players.loc[df_players['account_id'] == 9403474, 'fantasy_role'] = 2 # yamich
df_players.loc[df_players['account_id'] == 439345730, 'fantasy_role'] = 4 # reibl




## Make roles human readable

In [217]:
df_players['fantasy_role'] = df_players['fantasy_role'].astype(str)
df_players['fantasy_role'] = df_players['fantasy_role'].replace({'1.0': 'Core', '2.0': 'Support', '4.0': 'Mid'})

In [218]:
df_players['fantasy_role']

0        Support
1           Core
2           Core
3        Support
4            Mid
          ...   
15432    Support
15433        Mid
15434    Support
15435       Core
15436       Core
Name: fantasy_role, Length: 15437, dtype: object

In [219]:
df_players[df_players['fantasy_role'].isna()]

,match_id,match_start_time,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating,fantasy_score,rating_advantage


In [220]:
df_players_test = df_players.astype({
    'match_id': 'string',
    'account_id': 'string',
    'hero_id': 'string',
    'radiant_team_id': 'string',
    'dire_team_id': 'string',
    'team_id': 'string',
	'enemy_team_id': 'string',
    'valveId': 'string',
})

In [221]:
df_players.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15437 entries, 0 to 15436
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 15437 non-null  int64  
 1   match_start_time         15437 non-null  int64  
 2   match_start_time         15437 non-null  int64  
 3   account_id               15437 non-null  int64  
 4   name                     15437 non-null  object 
 5   fantasy_role             15437 non-null  object 
 6   hero_id                  15437 non-null  int64  
 7   hero_variant             15437 non-null  object 
 8   isRadiant                15437 non-null  bool   
 9   radiant_team_id          15437 non-null  int64  
 10  dire_team_id             15437 non-null  int64  
 11  tournament_start_date    15437 non-null  object 
 12  duration                 15437 non-null  int64  
 13  win                      15437 non-null  int64  
 14  kills                 

In [222]:
df_players = df_players.loc[:, ~df_players.columns.duplicated()] 

# Feature Engineering

## rating_advantage

In [223]:
df_players[~df_players['rating_advantage'].isna()].info()
df_players_clean = df_players[~df_players['rating_advantage'].isna()]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12741 entries, 0 to 15426
Data columns (total 40 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   match_id                 12741 non-null  int64  
 1   match_start_time         12741 non-null  int64  
 2   account_id               12741 non-null  int64  
 3   name                     12741 non-null  object 
 4   fantasy_role             12741 non-null  object 
 5   hero_id                  12741 non-null  int64  
 6   hero_variant             12741 non-null  object 
 7   isRadiant                12741 non-null  bool   
 8   radiant_team_id          12741 non-null  int64  
 9   dire_team_id             12741 non-null  int64  
 10  tournament_start_date    12741 non-null  object 
 11  duration                 12741 non-null  int64  
 12  win                      12741 non-null  int64  
 13  kills                    12741 non-null  int64  
 14  deaths                

## rolling_winrate

In [224]:
df_players = df_players.sort_values(['account_id', 'match_start_time'])

In [225]:
df_players['rolling_winrate_10'] = (
	df_players.groupby('account_id')['win']
	.apply(lambda x: x.shift().rolling(window=10, min_periods=1).mean()).reset_index(level=0, drop=True)
)

In [226]:
df_players['rolling_winrate_15'] = (
	df_players.groupby('account_id')['win']
	.apply(lambda x: x.shift().rolling(window=15, min_periods=1).mean()).reset_index(level=0, drop=True)
)

## rolling_fantasy_score

In [227]:
df_players['rolling_fantasy_score_10'] = (
	df_players.groupby('account_id')['fantasy_score']
	.apply(lambda x: x.shift().rolling(window=10, min_periods=1).mean()).reset_index(level=0, drop=True)
)

In [228]:
df_players['rolling_fantasy_score_15'] = (
	df_players.groupby('account_id')['fantasy_score']
	.apply(lambda x: x.shift().rolling(window=15, min_periods=1).mean()).reset_index(level=0, drop=True)
)

## rolling_enemy_glicko

In [229]:
df_players['rolling_enemy_glicko2_10'] = (
	df_players.groupby('account_id')['enemy_glicko2_rating']
	.apply(lambda x: x.shift().rolling(window=10, min_periods=1).mean()).reset_index(level=0, drop=True)
)

In [230]:
df_players['rolling_enemy_glicko2_15'] = (
	df_players.groupby('account_id')['enemy_glicko2_rating']
	.apply(lambda x: x.shift().rolling(window=15, min_periods=1).mean())
	.reset_index(level=0, drop=True)
)

In [231]:
df_players

,match_id,match_start_time,account_id,name,fantasy_role,hero_id,hero_variant,isRadiant,radiant_team_id,dire_team_id,tournament_start_date,duration,win,kills,deaths,last_hits,denies,gold_per_min,towers_killed,roshans_killed,teamfight_participation,obs_placed,camps_stacked,rune_pickups,firstblood_claimed,stuns,creeps_stacked,assists,xp_per_min,lane_efficiency,team_id,enemy_team_id,teamName,valveId,glicko2_rating,enemy_teamName,enemy_valveId,enemy_glicko2_rating,fantasy_score,rating_advantage,rolling_winrate_10,rolling_winrate_15,rolling_fantasy_score_10,rolling_fantasy_score_15,rolling_enemy_glicko2_10,rolling_enemy_glicko2_15
14216,8300451299,1747734901,9403474,yamich,Support,34,"34, 1",False,9467224,7554697,2025-05-19,1902,0,1,9,108,5,274,0,0,0.272727,3,3,1,1,0.000000,9,3,400,0.359337,7554697,9467224,Nigma Galaxy,7554697,1882.868495,Aurora Gaming,9467224,1942.349787,9.555182,-59.481293,0.5,0.533333,13.814227,14.313391,2047.501263,2006.848254
14186,8300502742,1747738740,9403474,yamich,Support,21,"21, 5",False,9467224,7554697,2025-05-19,2231,1,4,5,61,1,343,0,0,0.740741,6,1,1,0,65.098500,1,17,506,0.326192,7554697,9467224,Nigma Galaxy,7554697,1882.868495,Aurora Gaming,9467224,1942.349787,12.799147,-59.481293,0.6,0.466667,16.340131,15.452083,1917.222026,1945.018678
14046,8300798267,1747753863,9403474,yamich,Support,100,"100, 1",False,9780800,7554697,2025-05-19,1763,1,8,1,35,2,376,0,0,0.823529,5,1,2,0,51.900692,3,20,620,0.441593,7554697,9780800,Nigma Galaxy,7554697,1882.868495,NaN,NaN,NaN,14.528623,NaN,0.0,0.000000,7.868171,7.868171,1926.170888,1926.170888
14021,8300877198,1747757526,9403474,yamich,Support,100,"100, 1",True,7554697,9780800,2025-05-19,1401,1,1,3,17,1,274,0,0,0.444444,7,1,8,0,15.100049,8,11,419,0.348828,7554697,9780800,Nigma Galaxy,7554697,1882.868495,NaN,NaN,NaN,11.090336,NaN,0.5,0.466667,12.411325,12.890987,2010.886742,1979.958730
13881,8301640076,1747821304,9403474,yamich,Support,100,"100, 1",True,7554697,7732977,2025-05-19,1409,1,2,6,10,1,283,0,0,0.583333,4,0,6,0,9.333803,0,12,379,0.405618,7554697,7732977,Nigma Galaxy,7554697,1882.868495,BOOM Esports,7732977,1796.974895,8.115690,85.893600,0.2,0.266667,14.050052,15.055353,1978.435821,1974.270186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10830,8209412397,1741680134,1657135701,nesfeer,Core,53,"53, 2",True,9651814,7554697,2025-03-08,1914,0,0,6,310,3,532,1,0,0.333333,1,2,1,0,0.000000,4,4,617,0.851051,9651814,7554697,NaN,NaN,NaN,Nigma Galaxy,7554697,1889.225293,6.953000,NaN,0.6,0.600000,15.644959,15.589476,1983.117253,1942.213940
10811,8209457835,1741683998,1657135701,nesfeer,Core,70,"70, 2",False,7554697,9651814,2025-03-08,2388,1,8,2,374,8,727,3,2,0.562500,0,0,7,0,0.000000,0,10,905,0.887429,9651814,7554697,NaN,NaN,NaN,Nigma Galaxy,7554697,1889.225293,15.837500,NaN,0.5,0.533333,16.752621,16.802276,1939.252963,1922.096608
10776,8209507662,1741688073,1657135701,nesfeer,Core,49,"49, 3",True,9651814,7554697,2025-03-08,3663,1,12,5,554,7,692,6,2,0.600000,0,3,7,0,28.367176,7,9,915,0.790218,9651814,7554697,NaN,NaN,NaN,Nigma Galaxy,7554697,1889.225293,22.635359,NaN,0.5,0.400000,13.513187,14.386684,1987.501562,1961.985388
10678,8210666453,1741766444,1657135701,nesfeer,Core,102,"102, 2",True,9651814,9467224,2025-03-08,1981,0,6,3,256,7,536,2,1,0.571429,0,1,4,0,0.000000,3,2,536,0.854487,9651814,9467224,NaN,NaN,NaN,Aurora Gaming,9467224,1799.943451,11.975286,NaN,0.5,0.533333,15.714192,15.424902,1902.875067,1914.075073


# Write data to json files

In [232]:
df_players.to_json('data/curated/player_data.json')

In [233]:
df_players_clean = df_players.dropna().copy()

In [234]:
df_players_clean.to_json('data/curated/player_data_clean.json')